# Heart Disease Identification

import libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,plot_confusion_matrix
from sklearn import tree

import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
import copy
from sklearn.model_selection import KFold
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score

Read dataset and split into training and testing sets.

In [ ]:
data = pd.read_csv("../input/heart-disease-uci/heart.csv")

X = data.drop('target',axis=1)
y = data['target']

#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train.head(5)

In [ ]:
print("Samples:",len(y))
print("No disease:",list(y).count(0))
print("Disease:",list(y).count(1))

In [ ]:
def r2(a,b):
    this_correlation = np.corrcoef(a, b)[0,1]
    this_r2 = this_correlation**2
    return this_r2

function to plot confusion matrices

In [ ]:
def plot_confusion(title, model, X_train, y_train, X_test, y_test):
    svm_confusion_matrix = plot_confusion_matrix(model, X_train, y_train,
                      display_labels=['No Heart Disease','Heart Disease'],
                      cmap=plt.cm.YlOrBr)
    svm_confusion_matrix.ax_.set_title(title + " Confusion Matrix (Training Set)")
    plt.show()

    svm_confusion_matrix = plot_confusion_matrix(model, X_test, y_test,
                          display_labels=['No Heart Disease','Heart Disease'],
                          cmap=plt.cm.YlOrBr)
    svm_confusion_matrix.ax_.set_title(title + " Confusion Matrix (Testing Set)")
    plt.show()

function to draw learning curves

In [ ]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html#sphx-glr-auto-examples-model-selection-plot-learning-curve-py
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 1, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Predictive Accuracy")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True,verbose=2)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")
    print("CV Scores:",test_scores_mean)

    return plt

# Decision Tree

First, Create and display the untuned model:

In [ ]:
decision_tree = tree.DecisionTreeClassifier(max_depth=6, 
                                  criterion='gini',
                                  min_samples_leaf=1,
                                  min_samples_split=2,
                                  random_state=0)
decision_tree.fit(X_train, y_train)

y_train_pred = decision_tree.predict(X_train)
y_test_pred = decision_tree.predict(X_test)

print('Decision Tree Train Accuracy' , accuracy_score(y_train, y_train_pred))
print('Decision Tree Test Accuracy' , accuracy_score(y_test, y_test_pred))

cv = KFold(n_splits=10, random_state=0, shuffle=True)

fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(decision_tree, "Initial Decision Tree Learning Rate", 
                    X_train, y_train, axes=[axes], ylim=(0.4, 1.01),
                    cv=cv, n_jobs=4)

Gridsearch decision tree hyperparameters

In [ ]:
dt_params = {
    "criterion":['gini','entropy'],
    "max_depth":range(3,15),
    "min_samples_leaf":range(1,15),
    "min_samples_split":range(1,12),
    
}
decision_tree = tree.DecisionTreeClassifier()


grid = GridSearchCV(decision_tree,
                    param_grid = dt_params,
                    cv=10,
                    verbose=1,
                    n_jobs=-1
)
grid.fit(X_train,y_train)
print(grid.best_params_)
#{'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 11, 'min_samples_split': 2}

Run the Decision tree algorithm.

In [ ]:
start_time = time.time();
decision_tree = tree.DecisionTreeClassifier(max_depth=4, 
                                  criterion='entropy',
                                  min_samples_leaf=11,
                                  min_samples_split=2,
                                  class_weight={0: 1, 1: 3},
                                  random_state=0)
decision_tree.fit(X_train, y_train)
print("Training time:",time.time()-start_time)

start_time = time.time();
y_train_pred = decision_tree.predict(X_train)
y_test_pred = decision_tree.predict(X_test)
print("Prediction Time:",time.time()-start_time, "for",len(X_train) + len(X_test),"samples")


print('Decision Tree Train Accuracy' , accuracy_score(y_train, y_train_pred))
print("Decision Tree Train r2 score:",r2(y_train, y_train_pred))
print('Decision Tree Test Accuracy' , accuracy_score(y_test, y_test_pred))
print("Decision Tree Test r2 score:",r2(y_test, y_test_pred))
print("F1 SCORE:",round(f1_score(y_test, y_test_pred, average='micro'),3))

Show the Decision Tree learning rate

In [ ]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)

fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(decision_tree, "Final Decision Tree Learning Rate", X_train, y_train, axes=[axes], ylim=(0.4, 1.01),
                    cv=cv, n_jobs=4)

Show the Decision Tree confusion matrix

In [ ]:
plot_confusion("Decision Tree", decision_tree, X_train, y_train, X_test, y_test)

# Neural Network

Create and display a network with estimated parameters

In [ ]:
nn = MLPClassifier(activation='relu',
                   hidden_layer_sizes=(13,),
                   solver='lbfgs',
                   verbose=True,
                   max_iter=500,
                   random_state=1,
                   early_stopping=True)

nn.fit(X_train, y_train)
y_test_pred = nn.predict(X_test)
y_train_pred = nn.predict(X_train)

print('Neural Network Train Accuracy' , accuracy_score(y_train, y_train_pred))
print('Neural Network Test Accuracy' , accuracy_score(y_test, y_test_pred))

cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(nn, "Initial Neural Network Learning Rate", X_train, y_train, axes=[axes], ylim=(0.65, 1.01),
                    cv=cv, n_jobs=4)

Tune the Neural Network hyperparameters with GridSearch

In [ ]:
nn_params = {
    "hidden_layer_sizes":[(13,),(),(13,6),(6,)],
    "activation":['identity', 'logistic', 'tanh', 'relu'],
    "solver":['lbfgs', 'sgd', 'adam'],
    "verbose":[True],
    "max_iter":range(20,800,50)
}

nn = MLPClassifier()
grid = GridSearchCV(nn,
                    param_grid = nn_params,
                    cv=10,
                    verbose=1,
                    n_jobs=-1
)
grid.fit(X_train,y_train)
print(grid.best_params_)
#{'activation': 'identity', 'hidden_layer_sizes': (6,), 'max_iter': 120, 'solver': 'lbfgs', 'verbose': True}

Run the Neural Network

In [ ]:
start_time = time.time();
nn = MLPClassifier(activation='identity',
                   hidden_layer_sizes=(6,),
                   solver='lbfgs',
                   verbose=True,
                   max_iter=54,
                   random_state=1,
                   early_stopping=True)

nn.fit(X_train, y_train)
print("Execution Time:",time.time()-start_time)
start_time = time.time();

y_test_pred = nn.predict(X_test)
y_train_pred = nn.predict(X_train)
print("Prediction Time:",time.time()-start_time, "for",len(X_train) + len(X_test),"samples")

print('Neural Network Train Accuracy' , accuracy_score(y_train, y_train_pred))
print('Neural Network Train r2 score:',r2(y_train, y_train_pred))
print('Neural Network Test Accuracy' , accuracy_score(y_test, y_test_pred))
print("Neural Network Test r2 score:",r2(y_test, y_test_pred))
print("F1 SCORE:",round(f1_score(y_test, y_test_pred, average='micro'),3))

Calculate the Neural Network accuracy by iterations.

In [ ]:
nn_scores_train = [];
nn_scores = [];
for i in range(20):
    nn = MLPClassifier(activation='identity',
                   hidden_layer_sizes=(),
                   solver='lbfgs',
                   verbose=True,
                   max_iter=i*6+1,
                   random_state=1,
                   early_stopping=True)
    
    nn.fit(X_train,y_train);
    y_test_pred = nn.predict(X_test)
    nn_scores_train.append(accuracy_score(y_train, y_train_pred))
    nn_scores.append(accuracy_score(y_test, y_test_pred))
    print("I",i)
print("Training Scores:",nn_scores_train)
print("Testing Scores:",nn_scores)

In [ ]:
plt.plot(range(1,201,10),nn_scores_train,marker='o',label="Training Set")
plt.plot(range(1,201,10),nn_scores,marker='o',label="Testing Set")
plt.xlabel('Network Iterations')
plt.ylabel('Classification Accuracy')
plt.title("Neural Network Accuracy by Iteration Number")
plt.legend()
plt.show()

Plot Neural Network learning rate

In [ ]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(nn, "Final Neural Network Learning Rate", X_train, y_train, axes=[axes], ylim=(0.65, 1.01),
                    cv=cv, n_jobs=4)

Plot Neural Network confusion matrix

In [ ]:
plot_confusion("Neural Network", nn, X_train, y_train, X_test, y_test)

# Boosting

Create estimated model

In [ ]:
booster = AdaBoostClassifier(
    tree.DecisionTreeClassifier(max_depth=3, 
                                  criterion='gini',
                                  min_samples_leaf=1,
                                  min_samples_split=2,
                                  class_weight={0: 1, 1: 1},
                                  random_state=0),
    n_estimators=100,
    learning_rate=0.1,
    random_state=0) 
booster.fit(X_train, y_train)

y_test_pred = booster.predict(X_test)
y_train_pred = booster.predict(X_train)
print('Boosted Tree Train Accuracy' , accuracy_score(y_train, y_train_pred))
print('Boosted Tree Test Accuracy' , accuracy_score(y_test, y_test_pred))

cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(booster, "Initial ADA Booster Learning Rate", X_train, y_train, axes=[axes], ylim=(0.45, 1.05),
                    cv=cv, n_jobs=4)

Run Adaptive Boosting model

In [ ]:
start_time = time.time();
booster = AdaBoostClassifier(
    tree.DecisionTreeClassifier(max_depth=5, 
                                  criterion='entropy',
                                  min_samples_leaf=11,
                                  min_samples_split=2,
                                  class_weight={0: 1, 1: 3},
                                  random_state=0),
    n_estimators=500,
    learning_rate=0.75,
    random_state=0) 
booster.fit(X_train, y_train)
print("Execution Time:",time.time()-start_time)
start_time = time.time();

y_test_pred = booster.predict(X_test)
y_train_pred = booster.predict(X_train)
print("Prediction Time:",time.time()-start_time, "for",len(X_train) + len(X_test),"samples")

print('Boosted Tree Train Accuracy' , accuracy_score(y_train, y_train_pred))
print("Boosted Tree Train r2 score:",r2(y_train, y_train_pred))
print('Boosted Tree Test Accuracy' , accuracy_score(y_test, y_test_pred))
print("Boosted Tree Test r2 score:",r2(y_test, y_test_pred))
print("F1 SCORE:",round(f1_score(y_test, y_test_pred, average='micro'),3))

Calculate model accuracy by iteration

In [ ]:
start_time = time.time();
train_accuracies = [];
test_accuracies = [];
for i in range(1,1001,50):
    booster = AdaBoostClassifier(
        tree.DecisionTreeClassifier(max_depth=5, 
                                      criterion='entropy',
                                      min_samples_leaf=11,
                                      min_samples_split=2,
                                      class_weight={0: 1, 1: 3},
                                      random_state=0),
        n_estimators=i,
        learning_rate=0.75,
        random_state=0) 
    booster.fit(X_train, y_train)
    y_test_pred = booster.predict(X_test)
    y_train_pred = booster.predict(X_train)
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    test_accuracies.append(accuracy_score(y_test, y_test_pred))
    print(i,end=" ")
print("Training Scores:",train_accuracies)
print("Testing Scores:",test_accuracies)

Draw Boosting Accuracy by estimator count

In [ ]:
plt.plot(range(1,1001,50),train_accuracies,marker='o',label="Training Set")
plt.plot(range(1,1001,50),test_accuracies,marker='o',label="Testing Set")
plt.xlabel('Estimators')
plt.ylabel('Classification Accuracy')
plt.title("Adaptive Boosting Accuracy by Estimator Count")
plt.legend()
plt.show()

Draw chart for Ada Booster learning rate

In [ ]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(booster, "ADA Booster Learning Rate", X_train, y_train, axes=[axes], ylim=(0.45, 1.05),
                    cv=cv, n_jobs=4)

Draw decision tree for Adaptive Boosting model

In [ ]:
plot_confusion("Boosted Decision Tree", booster, X_train, y_train, X_test, y_test)

# SVM

Use GridSearch to tune Poly kernel SVM hyperparameters.

In [ ]:
svm_params1 = {
    "kernel":['poly'],
    "degree":range(1,9),
    "C":[4000,6000,8000, 12000,16000]
}

svm = SVC()
grid = GridSearchCV(svm,
                    param_grid = svm_params1,
                    cv=10,
                    verbose=1,
                    n_jobs=-1
)
grid.fit(X_train,y_train)
print(grid.best_params_)
print(grid.best_score_)
#{'C': 8000, 'degree': 2, 'kernel': 'poly'}

In [ ]:
svm_params1 = {
    "kernel":['rbf'],
    "gamma":['scale','auto']
}

svm = SVC()
grid = GridSearchCV(svm,
                    param_grid = svm_params1,
                    cv=10,
                    verbose=1,
                    n_jobs=-1
)
grid.fit(X_train,y_train)
print(grid.best_params_)
#{'gamma': 'scale', 'kernel': 'rbf'}

In [ ]:
start_time = time.time();
svm = SVC(kernel='rbf',
          gamma='scale',
          random_state=0)
svm.fit(X_train, y_train)

print("Execution Time:",time.time()-start_time)

start_time = time.time();
y_train_pred = svm.predict(X_train)
y_test_pred = svm.predict(X_test)
print("Prediction Time:",time.time()-start_time, "for",len(X_train) + len(X_test),"samples")

print("\nUsing RBF Kernel:")
print('SVM Train Accuracy' , accuracy_score(y_train, y_train_pred))
print("SVM Train r2 score:",r2(y_train, y_train_pred))
print('SVM Test Accuracy' , accuracy_score(y_test, y_test_pred))
print("SVM Test r2 score:",r2(y_test, y_test_pred))
print("F1 SCORE:",round(f1_score(y_test, y_test_pred, average='micro'),3))

In [ ]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(svm, "RBF Learning Rate", X_train, y_train, axes=[axes], ylim=(0.4, 0.9),
                    cv=cv, n_jobs=4)

In [ ]:
plot_confusion("SVM, Using RBF Kernel", svm, X_train, y_train, X_test, y_test)

Estimate poly kernel:

In [ ]:
svm = SVC(kernel='poly',
          C=1,
          degree=3,
          random_state=0)
svm.fit(X_train, y_train)

y_train_pred = svm.predict(X_train)
y_test_pred = svm.predict(X_test)

print("Using Poly Kernel:")
print('SVM Train Accuracy' , accuracy_score(y_train, y_train_pred))
print('SVM Test Accuracy' , accuracy_score(y_test, y_test_pred))

cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(svm, "Initial Poly Learning Rate", X_train, y_train, axes=[axes], ylim=(0.45, 0.8),
                    cv=cv, n_jobs=4)

In [ ]:
start_time = time.time();
svm = SVC(kernel='poly',
          C=8000,
          degree=2,
          class_weight={0: 1, 1: 5},
          random_state=0)
svm.fit(X_train, y_train)
print("Execution Time:",time.time()-start_time)

start_time = time.time();
y_train_pred = svm.predict(X_train)
y_test_pred = svm.predict(X_test)
print("Prediction Time:",time.time()-start_time, "for",len(X_train) + len(X_test),"samples")

print("Using Poly Kernel:")
print('SVM Train Accuracy' , accuracy_score(y_train, y_train_pred))
print("SVM Train r2 score:",r2(y_train, y_train_pred))
print('SVM Test Accuracy' , accuracy_score(y_test, y_test_pred))
print("SVM Test r2 score:",r2(y_test, y_test_pred))
print("F1 SCORE:",round(f1_score(y_test, y_test_pred, average='micro'),3))

In [ ]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(svm, "Poly Learning Rate", X_train, y_train, axes=[axes], ylim=(0.65, 1.01),
                    cv=cv, n_jobs=4)

In [ ]:
plot_confusion("SVM, Using Poly Kernel", svm, X_train, y_train, X_test, y_test)

# KNN Model

In [ ]:
knn_params = {
    "n_neighbors":range(1,10),
    "leaf_size":range(1,15),
    "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
}

knn = KNeighborsClassifier()
grid = GridSearchCV(knn,
                    param_grid = knn_params,
                    cv=10,
                    verbose=1,
                    n_jobs=-1
)
grid.fit(X_train,y_train)
print(grid.best_params_)
#{'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 3}

Estimate KNN model:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10,
                           algorithm='auto',
                           leaf_size=10)
knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

print('KNN Train Accuracy' , accuracy_score(y_train, y_train_pred))
print('KNN Test Accuracy' , accuracy_score(y_test, y_test_pred))

cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(knn, "Initial KNN Learning Rate", X_train, y_train, axes=[axes], ylim=(0.45, 0.85),
                    cv=cv, n_jobs=4)

In [ ]:
start_time = time.time();
knn = KNeighborsClassifier(n_neighbors=3,
                           algorithm='auto',
                           leaf_size=1)
knn.fit(X_train, y_train)

print("Execution Time:",time.time()-start_time)


start_time = time.time();
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

print("Prediction Time:",time.time()-start_time, "for",len(X_train) + len(X_test),"samples")

print("K = 4")
print('Decision Tree Train Accuracy' , accuracy_score(y_train, y_train_pred))
print("Decision Tree Train r2 score:",r2(y_train, y_train_pred))
print('KNN Test Accuracy' , accuracy_score(y_test, y_test_pred))
print("KNN Test r2 score:",r2(y_test, y_test_pred))
print("F1 SCORE:",round(f1_score(y_test, y_test_pred, average='micro'),3))

In [ ]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)
fig, axes = plt.subplots(1,1, figsize=(8, 5))
plot_learning_curve(knn, "KNN Learning Rate", X_train, y_train, axes=[axes], ylim=(0.45, 0.95),
                    cv=cv, n_jobs=4)

In [ ]:
plot_confusion("K Nearest Neighbors", knn, X_train, y_train, X_test, y_test)